# 09 | 语义检索，利用 Embedding 优化你的搜索功能

## 让 AI 生成实验数据

In [21]:

import openai, os
import pandas as pd

openai.api_key = os.environ.get("OPENAI_API_KEY")
COMPLETION_MODEL = "text-davinci-003"

def generate_data_by_prompt(prompt):
    response = openai.Completion.create(
        engine=COMPLETION_MODEL,
        prompt=prompt,
        temperature=0.5,
        max_tokens=2048,
        top_p=1,
    )
    return response.choices[0].text

prompt = """请你生成 50 条淘宝网里的商品的标题，每条在30个字左右，品类是3C数码产品，标题里往往也会有一些促销类的信息，每行一条。"""
data = generate_data_by_prompt(prompt)


product_names = data.strip().split('\n')
df = pd.DataFrame({'product_name': product_names})
df.head()

df.product_name = df.product_name.apply(lambda x: x.split('.')[1].strip())
df.head()


clothes_prompt = """请你生成 50 条淘宝网里的商品的标题，每条在30个字左右，品类是女性的服饰箱包等等，标题里往往也会有一些促销类的信息，每行一条。"""
clothes_data = generate_data_by_prompt(clothes_prompt)
clothes_product_names = clothes_data.strip().split('\n')
clothes_df = pd.DataFrame({'product_name': clothes_product_names})
clothes_df.product_name = clothes_df.product_name.apply(lambda x: x.split('.')[1].strip())
clothes_df.head()

df = pd.concat([df, clothes_df], axis=0)
df = df.reset_index(drop=True)

cereals_prompt = """请你生成 50 条淘宝网里的商品的标题，每条在30个字左右，品类是大米、谷物等等，标题里往往也会有一些促销类的信息，每行一条。"""
cereals_data = generate_data_by_prompt(cereals_prompt)
cereals_product_names = cereals_data.strip().split('\n')
cereals_df = pd.DataFrame({'product_name': cereals_product_names})
cereals_df.product_name = cereals_df.product_name.apply(lambda x: x.split('.')[1].strip())
cereals_df.head()

df = pd.concat([df, cereals_df], axis=0)
df = df.reset_index(drop=True)


display(df)

,product_name
0,Apple/苹果 MacBook Pro 16英寸笔记本电脑，超值享受！
1,小米/MI 10 Pro 5G手机，双11特惠，抢购必看！
2,三星/SAMSUNG Galaxy S20 Ultra 5G旗舰版，购机更优惠！
3,华为/HUAWEI Mate 40 Pro 5G，抢鲜体验，新品上市！
4,荣耀/HONOR MagicBook Pro，双11狂欢，高性价比！
...,...
139,【满减优惠】小麦粒5斤装
140,【限量抢购】米糠粉5斤装
141,【买2送1】玉米片粉5斤装
142,【限时特价】红豆米粉5斤装


## 通过 Embedding 进行语义搜索


In [22]:

from openai.embeddings_utils import get_embeddings
import openai, os, backoff

openai.api_key = os.environ.get("OPENAI_API_KEY")
embedding_model = "text-embedding-ada-002"

batch_size = 100

@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def get_embeddings_with_backoff(prompts, engine):
    embeddings = []
    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i+batch_size]
        embeddings += get_embeddings(list_of_text=batch, engine=engine)
    return embeddings

prompts = df.product_name.tolist()
prompt_batches = [prompts[i:i+batch_size] for i in range(0, len(prompts), batch_size)]

embeddings = []
for batch in prompt_batches:
    batch_embeddings = get_embeddings_with_backoff(prompts=batch, engine=embedding_model)
    embeddings += batch_embeddings

df["embedding"] = embeddings
df.to_parquet("data/taobao_product_title.parquet", index=False)


定义一个 search_product 的搜索函数，接受三个参数，一个 df 代表用于搜索的数据源，一个 query 代表用于搜索的搜索词，然后一个 n 代表搜索返回多少条记录。而这个函数就干了这样三件事情。
1. 调用 OpenAI 的 API 将搜索词也转换为 Embedding。
2. 将这个 Embedding 和 DataFrame 里的每一个 Embedding 都计算一下余弦距离。
3. 根据余弦相似度去排序，返回距离最近的 n 个标题。

In [26]:

from openai.embeddings_utils import get_embedding, cosine_similarity

# search through the reviews for a specific product
def search_product(df, query, n=3, pprint=True):
    product_embedding = get_embedding(
        query,
        engine=embedding_model
    )
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .product_name
    )
    if pprint:
        for r in results:
            print(r)
    return results

# results = search_product(df, "自然淡雅背包", n=3)
results = search_product(df, "粮食", n=3)

【限时特价】花生米粉5斤装
【月末特惠】玉米粒5斤装
【月末特惠】大麦米5斤装


## 利用 Embedding 信息进行商品推荐的冷启动

In [27]:

def recommend_product(df, product_name, n=3, pprint=True):
    product_embedding = df[df['product_name'] == product_name].iloc[0].embedding
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .product_name
    )
    if pprint:
        for r in results:
            print(r)
    return results

results = recommend_product(df, "【月末特惠】玉米粒5斤装", n=3)
# results = recommend_product(df, "【买一送一】精致小羊皮双肩包", n=3)

【月末特惠】玉米粒5斤装
【月末特惠】大麦米5斤装
【月末特惠】芝麻米5斤装


## 通过 FAISS 加速搜索过程

安装依赖
```
conda activate py310
conda install -c conda-forge faiss
# MacOS ARM 上只能安装 faiss-cpu
pip3 install faiss-cpu
pip3 install numpy --upgrade
```

In [28]:

import faiss
import numpy as np

def load_embeddings_to_faiss(df):
    embeddings = np.array(df['embedding'].tolist()).astype('float32')
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index

index = load_embeddings_to_faiss(df)

In [29]:

def search_index(index, df, query, k=5):
    query_vector = np.array(get_embedding(query, engine=embedding_model)).reshape(1, -1).astype('float32')
    distances, indexes = index.search(query_vector, k)

    results = []
    for i in range(len(indexes)):
        product_names = df.iloc[indexes[i]]['product_name'].values.tolist()
        results.append((distances[i], product_names))    
    return results

products = search_index(index, df, "自然淡雅背包", k=3)

for distances, product_names in products:
    for i in range(len(distances)):
        print(product_names[i], distances[i])
        

【新品】精致小巧真皮双肩包 0.21052608
【特惠】精致小巧真皮双肩包 0.21205075
【热卖】小巧精致真皮双肩包 0.22003332
